#### Attention:
Ceci ne prend pas en compte les départements d'outre mer mais uniquement la métrople !! (je n'ai pas trouvé de carte fonctionnel)

In [171]:
!pip install altair
import altair as alt 
alt.data_transformers.enable('json')


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


DataTransformerRegistry.enable('json')

In [ ]:
# imports
import pandas as pd
# import matplotlib.pyplot as plt
import plotly.graph_objs as go
import numpy as np

In [140]:
df = pd.read_csv("data/dpt2020.csv", sep=";")

# Vérifier si '2A' ou '2B' sont présents
print("'2A' présent ?", '2A' in df['dpt'].unique())
print("'2B' présent ?", '2B' in df['dpt'].unique())

'2A' présent ? False
'2B' présent ? False


In [141]:
print(f"Initial shape is {df.shape}")

# supp years and dpt = XXXX and transform to int
to_clean=['annais', 'dpt']
for col in to_clean:
    #df[col] = df[col].astype(str).str.strip()
    
    # Supprimer les lignes où la colonne n'est pas strictement numérique
    df = df[df[col].str.isnumeric()]
    
    if col == 'annais':
        df[col] = df[col].astype(int)
        
    else:
    # Reforcer le type string natif (facultatif mais propre)
        df[col] = df[col].astype("string")
        
        ### Ce n'est pas supported par la carte que nous avons ###
        # Transformation pour la corse => tout est transformé en 2A
        df.loc[df['dpt'] == '20', 'dpt'] = '2A'
        # Exclusion des DOM_TOM
        df = df[~df['dpt'].isin(['971', '972', '973', '974'])]
    print(df[col].dtype)
    print(f"Shape is now {df.shape}")

Initial shape is (3727553, 5)
int32
Shape is now (3690309, 5)
string
Shape is now (3492154, 5)


In [144]:
# Supprimer les prénoms "_PRENOMS_RARES"
df = df[df["preusuel"] != "_PRENOMS_RARES"]
# Ajouter une colonne de longueur de prénom
df["longueur"] = df["preusuel"].str.len()
# Supprimer les prénoms de longueur 1
df = df[df["longueur"] > 1]
print(f"Shape is now {df.shape}")

Shape is now (3471072, 6)


In [145]:
grouped = df.groupby(['annais', 'dpt', 'sexe', 'preusuel'], as_index=False)['nombre'].sum()
top_names = grouped.sort_values('nombre', ascending=False).groupby(['annais', 'dpt', 'sexe']).first().reset_index()

In [146]:
top_names = top_names.rename(columns={'preusuel': 'prenom'})

In [165]:
top_names['dpt'] = top_names['dpt'].str.zfill(2)

In [152]:
url = "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"
departements = alt.topo_feature(url, "features")

In [153]:
### TEST POUR LES CORRESPONDANCES DES DEPARTEMENTS ###

import requests
response = requests.get(url)
geojson = response.json()

# Extraire tous les codes de département du GeoJSON
geojson_codes = {feature['properties']['code'] for feature in geojson['features']}

# Extraire les codes du dataset
top_names_codes = set(top_names['dpt'].unique())

print("Codes présents dans GeoJSON mais absents de top_names :", geojson_codes - top_names_codes)
print("Codes présents dans top_names mais absents du GeoJSON :", top_names_codes - geojson_codes)
print(f"Correspondances exactes : {len(geojson_codes & top_names_codes)} / {len(geojson_codes)}")

Codes présents dans GeoJSON mais absents de top_names : {'2B'}
Codes présents dans top_names mais absents du GeoJSON : set()
Correspondances exactes : 95 / 96


In [162]:
df.head()

,sexe,preusuel,annais,dpt,nombre,longueur
10885,1,AADIL,1983,84,3,5
10886,1,AADIL,1992,92,3,5
10888,1,AAHIL,2016,95,3,5
10892,1,AARON,1962,75,3,5
10893,1,AARON,1976,75,3,5


In [176]:
top_names.groupby('dpt').size().max() == 1

False

In [ ]:
import altair as alt

# Paramètre pour l'année (avec un slider)
year_param = alt.param(
    name="année",
    bind=alt.binding_range(
        min=top_names['annais'].min(),
        max=top_names['annais'].max(),
        step=1,
        name="Année :"
    ),
    value=2000
)

# Paramètre pour le sexe
gender_param = alt.param(
    name="sexe",
    bind=alt.binding_radio(
        options=['1', '2'],
        labels=['♂ Homme', '♀ Femme'],
        name="Sexe :"
    ),
    value='1'
)

# Carte choroplèthe avec lookup + filtrage
carte = alt.Chart(departements).mark_geoshape(
    stroke='black',
    strokeWidth=0.5
).encode(
    tooltip=[
        alt.Tooltip('properties.nom:N', title='Département'),
        alt.Tooltip('prenom:N', title='Prénom')
    ],
    color=alt.Color('prenom:N', title='Prénom populaire', legend=None)
).transform_lookup(
    lookup='properties.code',
    from_=alt.LookupData(
        top_names,
        key='dpt',
        fields=['preusuel'] # ['annais', 'sexe', 'preusuel']
    )
)
""".transform_filter(
    (alt.datum.annais == year_param) & (alt.datum.sexe == gender_param)
)
.project(
    type='mercator'
).add_params(
    year_param,
    gender_param
).properties(
    width=700,
    height=600,
    title='Carte interactive des prénoms les plus populaires par département'
)
"""

carte


alt.Chart(...)